# Final Project

## Author: Cho Laam Yuen

## Load Data and Packages

In [1]:
#connect to my google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#change working directory
%cd /content/drive/My\ Drive/Adv\ ML/gr5074-final-project-ChoLaamY/
!pwd

/content/drive/My Drive/Adv ML/gr5074-final-project-ChoLaamY
/content/drive/My Drive/Adv ML/gr5074-final-project-ChoLaamY


In [3]:
#load my libraries
import sys
import time
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import os
import zipfile

from skimage.transform import resize
from sklearn.model_selection import train_test_split

from tensorflow.python.keras.utils import np_utils
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization
from tensorflow.python.keras.layers.convolutional import Conv2D, MaxPooling2D 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from tensorflow.keras.applications import VGG19, ResNet50, InceptionV3

In [4]:
#code taken from example code for Covid_Image_Classification (HW2)
#extracting all filenames iteratively
base_path = 'thumbnails3'
categories = ['Entertainment', 'Informative', 'Tech',]

# load file names to fnames list object
fnames = []
for category in categories:
    image_folder = os.path.join(base_path, category)
    file_names = os.listdir(image_folder)
    full_path = [os.path.join(image_folder, file_name) for file_name in file_names]
    fnames.append(full_path)

print('number of images for each category:', [len(f) for f in fnames])
print(fnames[0:1]) #examples of file names

number of images for each category: [813, 716, 774]
[['thumbnails3/Entertainment/l3ifRE7BQiM.jpg', 'thumbnails3/Entertainment/RuAQo97K-zE.jpg', 'thumbnails3/Entertainment/4nqJiBRNQuw.jpg', 'thumbnails3/Entertainment/2J6bJnwr1rw.jpg', 'thumbnails3/Entertainment/rRSOt9SpBMI.jpg', 'thumbnails3/Entertainment/AAtoia2BbjE.jpg', 'thumbnails3/Entertainment/nUSI5uUocho.jpg', 'thumbnails3/Entertainment/LHx63zUMLpM.jpg', 'thumbnails3/Entertainment/0U_Mpg59FSw.jpg', 'thumbnails3/Entertainment/uxhgrd6HWVw.jpg', 'thumbnails3/Entertainment/48DD3kekP3E.jpg', 'thumbnails3/Entertainment/qKFep4ekvx0.jpg', 'thumbnails3/Entertainment/IXgX_r-x-XE.jpg', 'thumbnails3/Entertainment/-pAOuR8s03Q.jpg', 'thumbnails3/Entertainment/Ew-3-8itpjc.jpg', 'thumbnails3/Entertainment/Bhos_P4a-JQ.jpg', 'thumbnails3/Entertainment/j5gy6QLq8wg.jpg', 'thumbnails3/Entertainment/U7Q3sKNuZA0.jpg', 'thumbnails3/Entertainment/Uk3-FxTF-BU.jpg', 'thumbnails3/Entertainment/iCd5Po4fPb0.jpg', 'thumbnails3/Entertainment/UT9a7EtxQOM.jpg', '

## Preprocess Data and Setup

In [5]:
#writing the preprocessor
# Import image, load to array of shape height, width, channels, then min/max transform.
# Write preprocessor that will match up with model's expected input shape.
# Uses opencv for image preprocessing

def preprocessor(data, shape=(320, 180)):
        """
        This function reads in images, resizes them to a fixed shape, and
        min/max transforms them, before converting feature values to float32
        for ONNX.
        
        params:
            data
                list of unprocessed images
                      
        returns:
            X
                numpy array of preprocessed image data
                  
        """
           
        import cv2
        import numpy as np

        "Resize a color image and min/max transform the image"
        img = cv2.imread(data) # Read in image from filepath.
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 reads in images in order of blue green and red, we reverse the order for ML.
        #grayscale image?  Use im_gray = cv2.imread('gray_image.png', cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, shape) # Change height and width of image.
        img = img / 255.0 # Min-max transform.  

        # Resize the images.
        X = np.array(img)
        #X = np.expand_dims(X, axis=0) # Expand dims to add "1" to object shape [1, h, w, channels] if needed.
        X = np.array(X, dtype=np.float32) # Final shape for onnx runtime.
        return X


In [7]:
#check whether the preprocessor worked
#(Height, Width, Channels)
preprocessor('thumbnails3/Tech/_O_kWL-YhZE.jpg').shape

(180, 320, 3)

In [8]:
#Import image files iteratively and preprocess them into array of correctly structured data

# Create list of file paths
image_filepaths=fnames[0]+fnames[1]+fnames[2]

# Iteratively import and preprocess data using map function

# map functions apply your preprocessor function one step at a time to each filepath
preprocessed_image_data=list(map(preprocessor,image_filepaths ))

# Object needs to be an array rather than a list for Keras (map returns to list object)
X= np.array(preprocessed_image_data) # Assigning to X to highlight that this represents feature input data for our model

In [9]:
#check length of files matched number of images in my dataset
len(image_filepaths)
print(len(X) ) #same number of images in dataset
print(X.shape ) #dimensions should be 180, 320, 3 for all images
print(X.min() ) #min value of every image is 0
print(X.max() ) #max value of every image is 1


2303
(2303, 180, 320, 3)
0.0
1.0


In [14]:
# Create y data made up of correctly ordered labels from file folders
from itertools import repeat

# 3 folders with the corresponding number of images in each folder

print('number of images for each category:', [len(f) for f in fnames])
Entertainment=list(repeat('Entertainment', 813))
Informative=list(repeat('Informative', 716))
Tech=list(repeat('Tech', 774))


#combine into single list of y labels
y_labels = Entertainment+Informative+Tech

#check length, same as X above
print(len(y_labels) )

# Need to one hot encode for Keras.  Let's use Pandas

import pandas as pd
y=pd.get_dummies(y_labels)

display(y)

number of images for each category: [813, 716, 774]
2303


,Entertainment,Informative,Tech
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
2298,0,0,1
2299,0,0,1
2300,0,0,1
2301,0,0,1


In [15]:
#train test split the dataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.20, random_state = 874920)

#check y_test set
y_test.sum()

Entertainment    163
Informative      143
Tech             155
dtype: int64

## Visualizing the Thumbnails

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random

pic1 = preprocessor('thumbnails/Automobile/-gZLTLmZBGM.jpg')
pic2 = preprocessor('thumbnails/Blog/3kJPxsELVAk.jpg')
pic3 = preprocessor('thumbnails/Comedy/0Kvw2BPKjz0.jpg')
pic4 = preprocessor('thumbnails/Entertainment/_qAJMXfL6o0.jpg')
pic5 = preprocessor('thumbnails/Food/A1S4pwewhyc.jpg')
pic6 = preprocessor('thumbnails/Informative/ElQdUHSXkME.jpg')
pic7 = preprocessor('thumbnails/News/Y4Zdx97A63s.jpg')
pic8 = preprocessor('thumbnails/Science/jAhjPd4uNFY.jpg')
pic9 = preprocessor('thumbnails/Tech/F0LNbWM-Hno.jpg')
pic10 = preprocessor('thumbnails/VideoGames/ADsp3UaLhl0.jpg')

#make a grid for the images
figure = plt.figure(figsize=(100, 100))
grid = ImageGrid(figure, 111,
                 nrows_ncols=(2, 5),
                 axes_pad=2,
                 )

for ax, pic in zip(grid, [pic1, pic2, pic3, pic4, pic5, pic6, pic7, pic8, pic9, pic10]):
  ax.imshow(pic)
  
#adjust the text size  
  plt.rc('font', size=60)

#create labels for images
  figure.text(0.12, 0.6, 'Automobile', color="black")
  figure.text(0.28, 0.6, 'Blog', color="black")
  figure.text(0.44, 0.6, 'Comedy', color="black")
  figure.text(0.6, 0.6, 'Entertainment', color="black")
  figure.text(0.76, 0.6, 'Food', color='black')
  figure.text(0.12, 0.5, 'Informative', color='black')
  figure.text(0.28, 0.5, 'News', color='black')
  figure.text(0.44, 0.5, 'Science', color='black')
  figure.text(0.6, 0.5, 'Tech', color='black')
  figure.text(0.76, 0.5, 'VideoGames', color='black')


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import numpy as np
import random

pic1 = preprocessor('thumbnails3/Entertainment/_qIRtFE6aIc.jpg')
pic2 = preprocessor('thumbnails3/Entertainment/8GxqvnQyaxs.jpg')
pic3 = preprocessor('thumbnails3/Entertainment/n4bucphC9r4.jpg')
pic4 = preprocessor('thumbnails3/Informative/UCA1A5GqCdQ.jpg')
pic5 = preprocessor('thumbnails3/Informative/el6No1wNKf0.jpg')
pic6 = preprocessor('thumbnails3/Informative/tJevBNQsKtU.jpg')
pic7 = preprocessor('thumbnails3/Tech/3dEfc9LL9bQ.jpg')
pic8 = preprocessor('thumbnails3/Tech/bCu0Z71QRF0.jpg')
pic9 = preprocessor('thumbnails3/Tech/DTBu4tigSDo.jpg')

#make a grid for the images
figure = plt.figure(figsize=(10, 10))
grid = ImageGrid(figure, 111,
                 nrows_ncols=(3, 3),
                 axes_pad=0.5,
                 )

for ax, pic in zip(grid, [pic1, pic2, pic3, pic4, pic5, pic6, pic7, pic8, pic9]):
  ax.imshow(pic)
  
#adjust the text size  
  plt.rc('font', size=60)

#create labels for images
  figure.text(0.12, 0.6, 'Entertainment', color="black")
  figure.text(0.28, 0.6, 'Entertainment', color="black")
  figure.text(0.44, 0.6, 'Entertainment', color="black")
  figure.text(0.6, 0.6, 'Informative', color="black")
  figure.text(0.76, 0.6, 'Informative', color='black')
  figure.text(0.12, 0.5, 'Informative', color='black')
  figure.text(0.28, 0.5, 'Tech', color='black')
  figure.text(0.44, 0.5, 'Tech', color='black')
  figure.text(0.6, 0.5, 'Tech', color='black')

## Model Experimentation

#### Model 1

In [ ]:
with tf.device('/device:GPU:0'):

  model1 = Sequential()
  model1.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu', input_shape=(180, 320, 3)))
  model1.add(Conv2D(filters=32, kernel_size=1, padding='same', activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
  model1.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model1.add(Conv2D(filters=32, kernel_size=1, padding='same', activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))
  model1.add(Conv2D(filters=32, kernel_size=3, padding='same', activation='relu'))
  model1.add(Conv2D(filters=32, kernel_size=1, padding='same', activation='relu'))
  model1.add(MaxPooling2D(pool_size=2))

  model1.add(Dropout(0.1))
  model1.add(Flatten())
  model1.add(Dense(16, activation='relu')) # One fully-connected layer
  model1.add(Dropout(0.1))

  model1.add(Dense(3, activation='softmax'))

  model1.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model1.fit(X_train, y_train, 
                    epochs = 5, verbose=1, validation_data=(X_test,y_test))

Epoch 1/5
58/58 [==============================] - 126s 2s/step - loss: 1.0949 - accuracy: 0.3730 - val_loss: 1.0876 - val_accuracy: 0.4403
Epoch 2/5
58/58 [==============================] - 125s 2s/step - loss: 1.0637 - accuracy: 0.4490 - val_loss: 1.0554 - val_accuracy: 0.4338
Epoch 3/5
58/58 [==============================] - 130s 2s/step - loss: 1.0244 - accuracy: 0.4810 - val_loss: 1.0661 - val_accuracy: 0.4642
Epoch 4/5
58/58 [==============================] - 132s 2s/step - loss: 0.9927 - accuracy: 0.5098 - val_loss: 0.9398 - val_accuracy: 0.5835
Epoch 5/5
58/58 [==============================] - 130s 2s/step - loss: 0.9463 - accuracy: 0.5353 - val_loss: 1.0015 - val_accuracy: 0.5315
